In [38]:
import gensim
import string
import nltk

from pyspark.sql.functions import col,lit, udf
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize

In [17]:
df_truncated = spark.read.json('/common/users/shared/cs543_fall22_group3/realnews/realnews_truncated')
df_truncated.show()

+--------------------+-------------------+------------+--------------------+--------------------+
|             authors|             domain|publish_date|             summary|               title|
+--------------------+-------------------+------------+--------------------+--------------------+
| [Reuters Editorial]|        reuters.com|  04-11-2013|U.S. investment b...|DEALTALK-Banks fi...|
|[Reuters Editoria...|        reuters.com|  04-11-2011|Mexican coffee pr...|Mexico coffee far...|
| [Reuters Editorial]|        reuters.com|  03-27-2013|Oil companies are...|Oil industry to s...|
|                  []|     sportsgrid.com|  07-23-2014|Adam Dunn … you a...|Fantasy Baseball:...|
|                  []|    staugustine.com|  01-01-2014|» County’s annual...|Teacher of the ye...|
|          [Sam Colt]|businessinsider.com|  06-20-2014|Dropcam will prob...|Google's Nest Is ...|
|     [Robert Ferris]|businessinsider.com|  02-28-2013|Wild Insects need...|Insects Are Disap...|
|[Fiona Macrae Sci..

In [42]:
# Remove punctuation, stop words, and lower case the letters
def remove_stopwords(line):
    tokens = word_tokenize(line)
    remove_stopwords = [t.lower() for t in tokens if not t in STOPWORDS or not t in string.punctuation]
    return ' '.join(remove_stopwords).strip(' ')

remove_stopwords_udf = udf(lambda z: remove_stopwords(z))
df_truncated.withColumn("cleaned_summary", remove_stopwords_udf(col("summary"))).show()

+--------------------+-------------------+------------+--------------------+--------------------+--------------------+
|             authors|             domain|publish_date|             summary|               title|     cleaned_summary|
+--------------------+-------------------+------------+--------------------+--------------------+--------------------+
| [Reuters Editorial]|        reuters.com|  04-11-2013|U.S. investment b...|DEALTALK-Banks fi...|u.s. investment b...|
|[Reuters Editoria...|        reuters.com|  04-11-2011|Mexican coffee pr...|Mexico coffee far...|mexican coffee pr...|
| [Reuters Editorial]|        reuters.com|  03-27-2013|Oil companies are...|Oil industry to s...|oil companies are...|
|                  []|     sportsgrid.com|  07-23-2014|Adam Dunn … you a...|Fantasy Baseball:...|adam dunn … you a...|
|                  []|    staugustine.com|  01-01-2014|» County’s annual...|Teacher of the ye...|» county ’ s annu...|
|          [Sam Colt]|businessinsider.com|  06-2